In [1]:
import datetime
import os
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import io


#download raw data from here: https://www.kaggle.com/netflix-inc/netflix-prize-data#README
#thanks to https://github.com/matthewkparker/Netflix_Recommender/blob/master/Code/01_Preprocessing.ipynb for help with data processing

#some resources below for enriching movie data
#most hopeful genre data https://www.igvita.com/2007/01/27/correlating-netflix-and-imdb-datasets/
#http://cns.bu.edu/~gsc/MovieGenre.html genre data
#alternatively can download data here: https://github.com/hadley/data-movies

def formatting(path):
    #Step 1
    df_raw = pd.read_csv(path, header=None, names=['user_id', 'rating', 'date'], usecols=[0, 1, 2])
    #Step 2
    tmp_movies = df_raw[df_raw['rating'].isna()]['user_id'].reset_index()
    movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]
    #Step 3
    shifted_movie_indices = deque(movie_indices)
    shifted_movie_indices.rotate(-1)
    #Step 4
    user_data = []
    for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
        if df_id_1<df_id_2:
            tmp_df = df_raw.loc[df_id_1+1:df_id_2-1].copy()
        else:
            tmp_df = df_raw.loc[df_id_1+1:].copy()
        tmp_df['movie'] = movie_id
        user_data.append(tmp_df)
    #Step 5
    df = pd.concat(user_data)
    print('done formatting')
    return df

path2file1 = r'C:/Users/fxi/Documents/Decision_Making_Under_Uncertainty/final_project/combined_data_1.txt'
path2file2 = r'C:/Users/fxi/Documents/Decision_Making_Under_Uncertainty/final_project/combined_data_2.txt'
path2file3 = r'C:/Users/fxi/Documents/Decision_Making_Under_Uncertainty/final_project/combined_data_3.txt'
path2file4 = r'C:/Users/fxi/Documents/Decision_Making_Under_Uncertainty/final_project/combined_data_4.txt'

df1 = formatting(path2file1)
df2 = formatting(path2file2)
df3 = formatting(path2file3)
df4 = formatting(path2file4)

done formatting
done formatting
done formatting
done formatting


In [2]:
def drop_threshold(threshold,column,df):
    filtered_df = df[df.groupby(column)[column].transform('size') >= threshold]
    print('This function dropped',len(df)-len(filtered_df),'rows by filtering on the',column,'column')
    return filtered_df

df_list = []
i = 1
for df in [df1,df2,df3,df4]:
    print('For Dataframe',i,':')
    #Step 1
    df.drop(columns=['date'],inplace = True)
    #Step 2
    dropped_user = drop_threshold(150,'user_id',df)
    df_list.append(drop_threshold(15000,'movie',dropped_user))
    i += 1
del df1
del df2
del df3
del df4


user_df = pd.concat(df_list)
user_df['user_id'] = pd.to_numeric(user_df['user_id'])
del df_list

user_df.to_csv('netflix_data_cleaned.csv')

For Dataframe 1 :
This function dropped 14870913 rows by filtering on the user_id column
This function dropped 5612461 rows by filtering on the movie column
For Dataframe 2 :
This function dropped 15290681 rows by filtering on the user_id column
This function dropped 6251804 rows by filtering on the movie column
For Dataframe 3 :
This function dropped 14808080 rows by filtering on the user_id column
This function dropped 5096893 rows by filtering on the movie column
For Dataframe 4 :
This function dropped 15757559 rows by filtering on the user_id column
This function dropped 5627403 rows by filtering on the movie column


In [40]:
movie_data = pd.read_csv("movie_titles.csv", header=None, encoding = "ISO-8859-1")
movie_data.columns = ['movie_id', 'release_year','movie_name']
netflix_data_cleaned=pd.merge(user_df, movie_data, left_on = 'movie', right_on = 'movie_id')
netflix_data_cleaned.head(5)

In [6]:
import scipy.io
import numpy as np

mat_path = r'C:/Users/fxi/Downloads/movieGenreBenchmark.mat'
#movieGenreData.mat'
genre_data = scipy.io.loadmat(mat_path)

movie_genre_mapping = pd.DataFrame(
    {'genre_id': genre_data['dataStructMovie'][0][0][1][:,0].tolist(),
     'movie_id':genre_data['dataStructMovie'][0][0][6][0].tolist()
    })    

k = [j[0] for j in genre_data['dataStructMovie'][0][0][8]] 

i=(list(range(1, len(k)+1)))

genre_names = pd.DataFrame(
    {'genre_id': i,
     'genre_name': k
    }) 

genre_mapping_cleaned=pd.merge(movie_genre_mapping, genre_names, left_on = 'genre_id', right_on = 'genre_id')
data=pd.merge(netflix_data_cleaned, genre_mapping_cleaned, left_on = 'movie_id', right_on = 'movie_id')

In [144]:
data.tail(100)

,user_id,rating,movie,movie_id,release_year,movie_name,genre_id,genre_name
14452014,1494205,4.0,17764,17764,1998.0,Shakespeare in Love,11,[Romance]
14452015,1602924,4.0,17764,17764,1998.0,Shakespeare in Love,11,[Romance]
14452016,255278,4.0,17764,17764,1998.0,Shakespeare in Love,11,[Romance]
14452017,2242161,3.0,17764,17764,1998.0,Shakespeare in Love,11,[Romance]
14452018,352258,5.0,17764,17764,1998.0,Shakespeare in Love,11,[Romance]
14452019,600160,3.0,17764,17764,1998.0,Shakespeare in Love,11,[Romance]
14452020,2087152,3.0,17764,17764,1998.0,Shakespeare in Love,11,[Romance]
14452021,1607300,4.0,17764,17764,1998.0,Shakespeare in Love,11,[Romance]
14452022,926461,4.0,17764,17764,1998.0,Shakespeare in Love,11,[Romance]
14452023,2553296,4.0,17764,17764,1998.0,Shakespeare in Love,11,[Romance]
